In [1]:
import pandas as pd

# 1) 데이터 로드
df = pd.read_excel('여성맞춤정책_통합데이터_v0.3.xlsx')

# 2) 카테고리 동의어 사전

category_keywords = {
    '1인가구': ['1인가구', '1인 가구', '독거'],
    '한부모': ['한부모', '조손가정', '미혼모'],
    '임신/출산/육아': ['출산', '육아', '신생아', '양육', '돌봄', '영유아', '보육', '임산부', '산모', '임부', '임신', '산부인과', '산전', '유산', '아동', '난임', '영아'],
    '초/중/고등학생': ['초등학생', '중학생', '고등학생', '초중고', '고교학비', '방과후학교', '방과후', '청소년', '초등', '중학', '고등', '초·중'],
    '대학(원)생': ['대학생', '대학'],
    '근로자/직장인': ['근로자', '직장인'],
    '구직/취업': ['취업', '구직', '일자리', '고용', '채용', '직업훈련'],
    '창업/자영업': ['예비창업자', '창업', '자영업', '소상공인', '예비창업자'],
    '고령자': ['고령자', '노인', '어르신', '경로자', '경로우대'],
    '신혼부부': ['신혼부부', '신혼', '부부'],
    '다문화': ['다문화', '결혼이민자', '외국인'],
    '농/어업인': ['농어촌','농업인', '어업인', '농가도우미', '농업경영체'],
    '취약계층': ['피해자', '희생자', '취약계층', '민주화운동', '북한이탈', '질환자', '질환', '희귀난치성', '국가유공자', '독립유공자', '장애', '저소득', '기초생활수급', '수급자', '차상위', '위기가구', '국민기초생활', '중위소득', '수급권자']
}

support_keywords = {
    '직접금전지원':   ['수당', '보조금', '지원금', '현금','지역화폐'],           
    '감면할인지원':   ['감면', '할인', '환급','수수료가 면제'],  
    '이용권':        ['이용권','쿠폰','바우처','상품권'],
    '교육지원':      ['교육비','학비','수강료','장학금','교육','학자','학업','방과 후'],
    '의료지원':      ['의료비','진료비','검진비','약품','의료','검사비','예방'],
    '주거지원':      ['주거비','임대료','전세','월세','주거 지원','주택','주거'],
    '교통지원':      ['교통비','교통카드','교통'],
    '생계지원':      ['생계비','식비','생활비','식료품비','우선임대'],
    '상담지원':      ['상담','멘토링','코칭'],
    '물품지원':      ['물품','용품','장비','현물'],
    '문화예술':      ['문화','예술','체험','공연','전시','도서 대여','여행'],
    '서비스':        ['서비스','돌봄','시설이용','무료입장','도우미','안심망'],
    '보험료지원':    ['보험료'],
    '법률지원':      ['법률', '면허정지','소송','변호']
}


# 4) 분류 함수
def classify(text, keyword_dict):
    text = text.lower()
    result = []
    for label, keywords in keyword_dict.items():
        for keyword in keywords:
            if keyword.lower() in text:
                result.append(label)
                break
    return result if result else ['기타']

# 5) 카테고리 분류
df['카테고리_분류'] = df.apply(lambda row: ','.join(
    classify(f"{row.get('제목', '')} {row.get('지원대상', '')} {row.get('지원내용', '')} {row.get('지역', '')}", category_keywords)
), axis=1)

# 6) 지원형태 분류
df['지원형태_분류'] = df.apply(lambda row: ','.join(
    classify(f"{row.get('제목', '')} {row.get('지원형태', '')} {row.get('지원내용', '')}", support_keywords)
), axis=1)

# 7) 결과 저장
cols = [
    '대상유형','지역','제목','detail_url','신청기간','신청방법',
    '접수기관','지원형태','지원대상','지원내용','문의처','기타',
    '지원형태_분류','카테고리_분류'
]
output_path = '여성맞춤정책_지원형태분류_동의어사전.xlsx'
df[cols].to_excel(output_path, index=False)

print(f"✅ 분류 완료: '{output_path}'")


✅ 분류 완료: '여성맞춤정책_지원형태분류_동의어사전.xlsx'


In [2]:
# -*- coding: utf-8 -*-
# ⚠️ 최초 1회 설치 필요:
# pip install -U scikit-learn sentence-transformers konlpy openpyxl rank_bm25 transformers faiss-cpu torch

import re
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, util, CrossEncoder
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

# ───────────────────────────────────────────────────────────
# 0. 설정 및 전처리
# ───────────────────────────────────────────────────────────
okt = Okt()
STOPWORDS = {
    '또는','그리고','및','하지만','그러나','또한','따라',
    '지원','위한','관련','있음','있는','되어','대한',
    '대해','의거','통해','경우','위해','사항'
}

def preprocess(text: str) -> str:
    if not isinstance(text, str):
        text = str(text)
    toks = okt.morphs(text, norm=True, stem=True)
    return " ".join(tok for tok in toks
                    if tok not in STOPWORDS and len(tok) > 1)

def rule_match(text: str, kw_dict: dict):
    """동의어 사전 기반으로 레이블과 매칭 키워드 모두 수집"""
    labels, kws = [], []
    raw = text.lower()
    for label, words in kw_dict.items():
        for w in words:
            if w.lower() in raw:
                labels.append(label)
                kws.append(w)
                break
    return labels, kws

# ───────────────────────────────────────────────────────────
# 1. 모델 로드
# ───────────────────────────────────────────────────────────
print("모델 로드를 시작합니다…")
sbert_model      = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
bi_encoder_model = SentenceTransformer('jhgan/ko-sroberta-multitask')
cross_encoder    = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
hyde_tokenizer   = PreTrainedTokenizerFast.from_pretrained(
    'gogamza/kobart-base-v2', padding_side='left')
hyde_model       = BartForConditionalGeneration.from_pretrained(
    'gogamza/kobart-base-v2')
hyde_tokenizer.add_special_tokens({'pad_token':'<pad>'})
hyde_model.resize_token_embeddings(len(hyde_tokenizer))
print("✅ 모델 준비 완료")

# ───────────────────────────────────────────────────────────
# 2. 동의어 사전
# ───────────────────────────────────────────────────────────
category_keywords = {
    '1인가구': ['1인가구', '1인 가구', '독거'],
    '한부모': ['한부모', '조손가정', '미혼모'],
    '임신/출산/육아': ['출산', '육아', '신생아', '양육', '돌봄', '영유아', '보육', '임산부', '산모', '임부', '임신', '산부인과', '산전', '유산', '아동', '난임', '영아'],
    '초/중/고등학생': ['초등학생', '중학생', '고등학생', '초중고', '고교학비', '방과후학교', '방과후', '청소년', '초등', '중학', '고등', '초·중'],
    '대학(원)생': ['대학생', '대학'],
    '근로자/직장인': ['근로자', '직장인'],
    '구직/취업': ['취업', '구직', '일자리', '고용', '채용', '직업훈련'],
    '창업/자영업': ['예비창업자', '창업', '자영업', '소상공인', '예비창업자'],
    '고령자': ['고령자', '노인', '어르신', '경로자', '경로우대'],
    '신혼부부': ['신혼부부', '신혼', '부부'],
    '다문화': ['다문화', '결혼이민자', '외국인'],
    '농/어업인': ['농어촌','농업인', '어업인', '농가도우미', '농업경영체'],
    '취약계층': ['피해자', '희생자', '취약계층', '민주화운동', '북한이탈', '질환자', '질환', '희귀난치성', '국가유공자', '독립유공자', '장애', '저소득', '기초생활수급', '수급자', '차상위', '위기가구', '국민기초생활', '중위소득', '수급권자']
}

support_keywords = {
    '직접금전지원':   ['수당', '보조금', '지원금', '현금','지역화폐'],           
    '감면할인지원':   ['감면', '할인', '환급','수수료가 면제'],  
    '이용권':        ['이용권','쿠폰','바우처','상품권'],
    '교육지원':      ['교육비','학비','수강료','장학금','교육','학자','학업','방과 후'],
    '의료지원':      ['의료비','진료비','검진비','약품','의료','검사비','예방'],
    '주거지원':      ['주거비','임대료','전세','월세','주거 지원','주택','주거'],
    '교통지원':      ['교통비','교통카드','교통'],
    '생계지원':      ['생계비','식비','생활비','식료품비','우선임대'],
    '상담지원':      ['상담','멘토링','코칭'],
    '물품지원':      ['물품','용품','장비','현물'],
    '문화예술':      ['문화','예술','체험','공연','전시','도서 대여','여행'],
    '서비스':        ['서비스','돌봄','시설이용','무료입장','도우미','안심망'],
    '보험료지원':    ['보험료'],
    '법률지원':      ['법률', '면허정지','소송','변호']
}


# ───────────────────────────────────────────────────────────
# 3. 하이브리드 매칭 함수 정의 (BM25 ZeroDivisionError 방지 포함)
# ───────────────────────────────────────────────────────────
def generate_pseudo_doc(text: str) -> str:
    try:
        inputs = hyde_tokenizer(text, return_tensors='pt',
                                 padding=True, truncation=True, max_length=512)
        out_ids = hyde_model.generate(input_ids=inputs.input_ids,
                                      attention_mask=inputs.attention_mask,
                                      max_length=128, num_beams=5,
                                      early_stopping=True)
        return hyde_tokenizer.decode(out_ids[0], skip_special_tokens=True)
    except:
        return ""

def hybrid_match(text: str, kw_dict: dict):
    labels = list(kw_dict.keys())
    protos = [' '.join(kw_dict[l]) for l in labels]
    tx_pre = preprocess(text)
    pr_pre = [preprocess(p) for p in protos]

    # 1) TF-IDF
    tf = TfidfVectorizer(ngram_range=(1,2))
    X  = tf.fit_transform(pr_pre + [tx_pre])
    tfidf_scores = (X[-1] @ X[:-1].T).toarray().flatten()

    # 2) BM25 (빈 리스트 제거 & try/except)
    tokenized = [p.split() for p in pr_pre]
    non_empty = [t for t in tokenized if t]
    try:
        if non_empty:
            bm25 = BM25Okapi(non_empty)
            scores_ne = bm25.get_scores(tx_pre.split())
            bm25_scores = np.zeros(len(labels), dtype=float)
            idx_map = [i for i, toks in enumerate(tokenized) if toks]
            for j, score in zip(idx_map, scores_ne):
                bm25_scores[j] = score
        else:
            bm25_scores = np.zeros(len(labels), dtype=float)
    except ZeroDivisionError:
        bm25_scores = np.zeros(len(labels), dtype=float)

    # 3) SBERT
    sbert_scores = util.cos_sim(
        sbert_model.encode([text], convert_to_tensor=True),
        sbert_model.encode(protos, convert_to_tensor=True)
    )[0].cpu().numpy()
    # 4) Bi-Encoder
    bi_scores = util.cos_sim(
        bi_encoder_model.encode([text], convert_to_tensor=True),
        bi_encoder_model.encode(protos, convert_to_tensor=True)
    )[0].cpu().numpy()
    # 5) Cross-Encoder
    cross_scores = cross_encoder.predict([[text, p] for p in protos])
    # 6) HyDE
    pseudo = generate_pseudo_doc(text)
    hyde_scores = (util.cos_sim(
        sbert_model.encode([pseudo], convert_to_tensor=True),
        sbert_model.encode(protos, convert_to_tensor=True)
    )[0].cpu().numpy() if pseudo else np.zeros(len(labels)))

    # 정규화 및 가중합
    all_scores = np.vstack([
        tfidf_scores, bm25_scores,
        sbert_scores, bi_scores,
        cross_scores, hyde_scores
    ])
    mn, mx = all_scores.min(axis=1, keepdims=True), all_scores.max(axis=1, keepdims=True)
    norm = (all_scores - mn) / (mx - mn + 1e-8)
    weights = np.array([0.1, 0.1, 0.25, 0.15, 0.3, 0.1], dtype=float)
    final = norm.T.dot(weights)
    idx = int(np.argmax(final))

    # 매칭 키워드
    matched = [w for w in kw_dict[labels[idx]] if w.lower() in text.lower()]
    if not matched:
        matched = list(set(tx_pre.split()) & set(pr_pre[idx].split()))

    return {
        'label': labels[idx],
        'model': 'hybrid_weighted',
        'matched_kw': ",".join(matched) or labels[idx],
        'tfidf_score': float(tfidf_scores[idx]),
        'bm25_score': float(bm25_scores[idx]),
        'sbert_score': float(sbert_scores[idx]),
        'bi_encoder_score': float(bi_scores[idx]),
        'cross_score': float(cross_scores[idx]),
        'hyde_score': float(hyde_scores[idx]),
        'final_score': float(final[idx])
    }

# ───────────────────────────────────────────────────────────
# 4. 전체 분류 & 후처리
# ───────────────────────────────────────────────────────────
if __name__ == '__main__':
    df = pd.read_excel('여성맞춤정책_통합데이터_v0.3.xlsx', engine='openpyxl')
    records = []
    used_cat_kws, used_sup_kws = set(), set()

    for _, row in df.iterrows():
        rec = row.to_dict()

        # --- 카테고리 룰/하이브리드 ---
        txt_cat = ' '.join(str(row[c]) for c in ['제목','지원대상','지원내용','지역'])
        labs_c, kws_c = rule_match(txt_cat, category_keywords)
        if labs_c:
            rec.update({
                'category_label':      ",".join(labs_c),
                'category_model':      'rule',
                'category_matched_kw': ",".join(kws_c),
                'category_final_score': 1.0
            })
            for s in ['tfidf','bm25','sbert','bi_encoder','cross','hyde']:
                rec[f'category_{s}_score'] = None
            used_cat_kws.update(kws_c)
        else:
            info = hybrid_match(txt_cat, category_keywords)
            for k, v in info.items():
                rec[f'category_{k}'] = v

        # --- 지원형태 룰 → 금액패턴(m1,m2,m_amt) → hybrid ---
        txt_sup = ' '.join(str(row[c]) for c in ['지원형태','지원내용','제목'])
        labs_s, kws_s = rule_match(txt_sup, support_keywords)
        if labs_s:
            rec.update({
                'support_label':      ",".join(labs_s),
                'support_model':      'rule',
                'support_matched_kw': ",".join(kws_s),
                'support_final_score': 1.0
            })
            for s in ['tfidf','bm25','sbert','bi_encoder','cross','hyde']:
                rec[f'support_{s}_score'] = None
            used_sup_kws.update(kws_s)
        else:
            m1    = re.search(r'\d+\s*(?:~\s*\d+)?\s*만\s*원', txt_sup)
            m2    = re.search(r'(\d{1,3}(?:,\d{3})+|\d+)\s*원', txt_sup)
            m_amt = re.search(r'\d+\s*(?:천만|만)?\s*원', txt_sup)
            if m1 or m2 or m_amt:
                matched = (m1 or m2 or m_amt).group()
                rec.update({
                    'support_label':      '직접금전지원',
                    'support_model':      'rule',
                    'support_matched_kw': matched,
                    'support_final_score': 1.0
                })
                for s in ['tfidf','bm25','sbert','bi_encoder','cross','hyde']:
                    rec[f'support_{s}_score'] = None
                used_sup_kws.add(matched)
            else:
                info = hybrid_match(txt_sup, support_keywords)
                for k, v in info.items():
                    rec[f'support_{k}'] = v

        records.append(rec)

        # …(위에서 레코드 수집까지 그대로)…

    out_df = pd.DataFrame(records)

    # --- 미사용 키워드 계산 & 출력 추가 ---
    all_cat_kw = set(sum(category_keywords.values(), []))
    all_sup_kw = set(sum(support_keywords.values(), []))
    unused_cat = sorted(all_cat_kw - used_cat_kws)
    unused_sup = sorted(all_sup_kw - used_sup_kws)
    print("미사용 카테고리 키워드:", unused_cat)
    print("미사용 지원형태 키워드:", unused_sup)

    # --- 후처리: 매칭키워드 없는 카테고리만 full fallback ---
    all_categories = list(category_keywords.keys())
    mask = out_df['category_matched_kw'].isna() | (out_df['category_matched_kw'].str.strip() == "")
    out_df.loc[mask, 'category_label']      = ",".join(all_categories)
    out_df.loc[mask, 'category_model']      = "fallback_all"
    out_df.loc[mask, 'category_matched_kw'] = "없음"

    # --- 결과 저장 ---
    out_df.to_excel('여성맞춤정책_분류+상세_v5.xlsx', index=False)
    print("✅ 후처리 완료: '여성맞춤정책_분류+상세_v5.xlsx'")


c:\Users\hyunj\anaconda3\envs\my_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


모델 로드를 시작합니다…


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.


✅ 모델 준비 완료
미사용 카테고리 키워드: []
미사용 지원형태 키워드: []
✅ 후처리 완료: '여성맞춤정책_분류+상세_v5.xlsx'


In [3]:
import pandas as pd

# ——— (이전 로직: records 리스트 생성, category_keywords 정의 등) ———
# out_df = pd.DataFrame(records)

# 예시로 out_df 가 이미 준비되었다고 가정합니다.
out_df = pd.DataFrame(records)

# 전체 카테고리 리스트(‘fallback_all’용)
all_categories = list(category_keywords.keys())

# ——— (1) 특정 행을 “전체 카테고리”로 덮어쓰기 ———
override_rows = [62, 342, 344,610,619,687, 912, 915, 916, 1221]

if 'index' in out_df.columns:
    mask = out_df['index'].isin(override_rows)
else:
    # 엑셀 원본 행번호 − 2 가 DataFrame 인덱스였다면
    mask = out_df.index.isin([r - 2 for r in override_rows])

out_df.loc[mask, 'category_label']      = ",".join(all_categories)
out_df.loc[mask, 'category_model']      = "fallback_all"
out_df.loc[mask, 'category_matched_kw'] = "없음"

# ——— (2-1) 819행만 코드상으로 내용 삭제후 “구직/취업”으로 덮어쓰기 ———
row_to_override = 819
if 'index' in out_df.columns:
    mask819 = out_df['index'] == row_to_override
else:
    mask819 = out_df.index == (row_to_override - 2)

out_df.loc[mask819, 'category_label'] = '구직/취업'

# ——— (2-2) 18행만 코드상으로 내용 삭제후 “취약계층”으로 덮어쓰기 ———
row_to_override = 18
if 'index' in out_df.columns:
    mask819 = out_df['index'] == row_to_override
else:
    mask819 = out_df.index == (row_to_override - 2)

out_df.loc[mask819, 'category_label'] = '취약계층'

# ——— (3) 불필요한 내부 컬럼들 삭제 ———
cols_to_drop = [
    'category_model', 
    'category_tfidf_score', 'category_bm25_score', 'category_sbert_score',
    'category_bi_encoder_score', 'category_cross_score', 'category_hyde_score', 'category_final_score',
    'support_model',  'support_final_score',
    'support_tfidf_score', 'support_bm25_score', 'support_sbert_score',
    'support_bi_encoder_score', 'support_cross_score', 'support_hyde_score'
    ,'category_matched_kw','support_matched_kw',
]
out_df = out_df.drop(columns=cols_to_drop, errors='ignore')

# ——— (4) 컬럼명 한글로 변경 ———
out_df = out_df.rename(columns={
    'category_label': '카테고리_분류',
    'support_label':  '지원형태_분류'
})

# ——— (5) 최종 결과 저장 ———
out_df.to_excel('여성맞춤정책_분류+상세_v6.xlsx', index=False)
print("✅ v6 저장 완료 (fallback_all + 819행 ‘구직/취업’ 덮어쓰기 반영)") 


✅ v6 저장 완료 (fallback_all + 819행 ‘구직/취업’ 덮어쓰기 반영)


In [4]:
import pandas as pd

# 1. 파일 불러오기
df1 = pd.read_excel('여성맞춤정책_지원형태분류_동의어사전.xlsx')
df2 = pd.read_excel('여성맞춤정책_분류+상세_v5.xlsx')

# 2. df1에서 '기타'인 행만 필터링
df_etc = df1[df1['카테고리_분류'] == '기타'].copy()

# 3. 같은 인덱스에서 df2의 카테고리 정보 가져오기
df_etc['category_label'] = df2.loc[df_etc.index, 'category_label']

# 4. 엑셀 인덱스 기준 번호 추가 (보기 편하게)
df_etc.insert(0, 'index', df_etc.index + 2)

# 5. 결과 출력
df_etc[['index', '제목', '카테고리_분류', 'category_label']]

,index,제목,카테고리_분류,category_label
60,62,횡성여성문화의 날 운영,기타,취약계층
340,342,보문골프장 입장요금 할인(여성),기타,취약계층
342,344,안동레이크골프장 입장요금 할인(여성),기타,농/어업인
364,366,풍진검사 지원,기타,임신/출산/육아
608,610,문화체육시설 이용요금 감면(여성보건),기타,취약계층
617,619,구로구립 여성합창단,기타,초/중/고등학생
685,687,용산문화체육센터 프로그램 할인(가임여성),기타,취약계층
693,695,24시간 여성안심망 「안심이」운영,기타,취약계층
849,851,맞춤형 기초생활 보장,기타,취약계층
910,912,여성기업 판로지원,기타,취약계층


In [5]:
import pandas as pd

# 1. 파일 불러오기
df1 = pd.read_excel('여성맞춤정책_지원형태분류_동의어사전.xlsx')
df2 = pd.read_excel('여성맞춤정책_분류+상세_v6.xlsx')

# 2. df1에서 '기타'인 행만 필터링
df_etc = df1[df1['카테고리_분류'] == '기타'].copy()

# 3. 같은 인덱스에서 df2의 카테고리 정보 가져오기
df_etc['카테고리_분류'] = df2.loc[df_etc.index, '카테고리_분류']

# 4. 엑셀 인덱스 기준 번호 추가 (보기 편하게)
df_etc.insert(0, 'index', df_etc.index + 2)

# 5. 결과 출력
df_etc[['index', '제목', '카테고리_분류', '카테고리_분류']]

,index,제목,카테고리_분류,카테고리_분류
60,62,횡성여성문화의 날 운영,"1인가구,한부모,임신/출산/육아,초/중/고등학생,대학(원)생,근로자/직장인,구직/취...","1인가구,한부모,임신/출산/육아,초/중/고등학생,대학(원)생,근로자/직장인,구직/취..."
340,342,보문골프장 입장요금 할인(여성),"1인가구,한부모,임신/출산/육아,초/중/고등학생,대학(원)생,근로자/직장인,구직/취...","1인가구,한부모,임신/출산/육아,초/중/고등학생,대학(원)생,근로자/직장인,구직/취..."
342,344,안동레이크골프장 입장요금 할인(여성),"1인가구,한부모,임신/출산/육아,초/중/고등학생,대학(원)생,근로자/직장인,구직/취...","1인가구,한부모,임신/출산/육아,초/중/고등학생,대학(원)생,근로자/직장인,구직/취..."
364,366,풍진검사 지원,임신/출산/육아,임신/출산/육아
608,610,문화체육시설 이용요금 감면(여성보건),"1인가구,한부모,임신/출산/육아,초/중/고등학생,대학(원)생,근로자/직장인,구직/취...","1인가구,한부모,임신/출산/육아,초/중/고등학생,대학(원)생,근로자/직장인,구직/취..."
617,619,구로구립 여성합창단,"1인가구,한부모,임신/출산/육아,초/중/고등학생,대학(원)생,근로자/직장인,구직/취...","1인가구,한부모,임신/출산/육아,초/중/고등학생,대학(원)생,근로자/직장인,구직/취..."
685,687,용산문화체육센터 프로그램 할인(가임여성),"1인가구,한부모,임신/출산/육아,초/중/고등학생,대학(원)생,근로자/직장인,구직/취...","1인가구,한부모,임신/출산/육아,초/중/고등학생,대학(원)생,근로자/직장인,구직/취..."
693,695,24시간 여성안심망 「안심이」운영,취약계층,취약계층
849,851,맞춤형 기초생활 보장,취약계층,취약계층
910,912,여성기업 판로지원,"1인가구,한부모,임신/출산/육아,초/중/고등학생,대학(원)생,근로자/직장인,구직/취...","1인가구,한부모,임신/출산/육아,초/중/고등학생,대학(원)생,근로자/직장인,구직/취..."


In [6]:
import pandas as pd

# 1. 파일 불러오기
df1 = pd.read_excel('여성맞춤정책_지원형태분류_동의어사전.xlsx')
df2 = pd.read_excel('여성맞춤정책_분류+상세_v6.xlsx')

# 2. df1에서 '기타'인 행만 필터링
df_etc = df1[df1['지원형태_분류'] == '기타'].copy()

# 3. 같은 인덱스에서 df2의 카테고리 정보 가져오기
df_etc['support_label'] = df2.loc[df_etc.index, '지원형태_분류']

# 4. 엑셀 인덱스 기준 번호 추가 (보기 편하게)
df_etc.insert(0, 'index', df_etc.index + 2)

# 5. 결과 출력
df_etc[['index', '제목', '지원형태_분류', 'support_label']]

,index,제목,지원형태_분류,support_label
62,64,2025년 경기임산부 친환경농산물 지원,기타,직접금전지원
749,751,결식아동 급식지원,기타,직접금전지원
798,800,결식아동 급식지원,기타,직접금전지원
832,834,근로장려금(2025년 신청),기타,직접금전지원
864,866,미소금융,기타,직접금전지원
900,902,양육비 선지급제,기타,직접금전지원
939,941,자녀장려금(2025년 신청),기타,직접금전지원
987,989,한부모 근로자 육아휴직급여,기타,직접금전지원
994,996,한시적 양육비 긴급지원,기타,직접금전지원
1024,1026,정보취약계층 대상 PC 정비비용 지원,기타,직접금전지원


In [7]:
# import pandas as pd

# # 1. 데이터 로드
# df = pd.read_excel('여성맞춤정책_분류+상세_v6.xlsx')

# # 2. 키워드 목록
# search_keywords = ['독거']

# # 3. 검색 대상 열들 결합
# df['_검색대상문장'] = (
#     df['제목'].fillna('') + ' ' +
#     df['지원내용'].fillna('') + ' ' +
#     df['지원대상'].fillna('')
# ).str.lower()

# # 4. 키워드 매칭 함수
# def extract_matching_keywords(text):
#     matched = [kw for kw in search_keywords if kw.lower() in text]
#     return ','.join(matched) if matched else ''

# df['매칭된_검색어'] = df['_검색대상문장'].apply(extract_matching_keywords)

# # 5. 매칭된 행 필터링 + 원래 인덱스값 + 2 추가
# filtered_df = df[df['매칭된_검색어'] != ''].copy()
# filtered_df['index'] = filtered_df.index + 2  # 원하는 값

# # 6. 불필요한 열 제거
# filtered_df.drop(columns=['_검색대상문장'], inplace=True)

# # 7. 불필요한 열 제거
# cols_to_drop = ['_검색대상문장', 'detail_url', '신청기간', '신청방법', '접수기관', '지원형태', '문의처', '기타']
# filtered_df.drop(columns=[col for col in cols_to_drop if col in filtered_df.columns], inplace=True)

# # 7. '인덱스+2' 열을 맨 앞으로 배치
# first_col = 'index'
# cols = [first_col] + [col for col in filtered_df.columns if col != first_col]
# filtered_df = filtered_df[cols]

# # 8. 엑셀로 저장
# filtered_df.to_excel('여성맞춤정책_비교_검색결과.xlsx', index=False)
# print("✅ 검색 완료! 'index+2' 컬럼 포함 결과가 저장되었습니다.")
